In [1]:
# Load libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from time import time
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
import warnings
import math
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF
import pickle
warnings.filterwarnings('ignore')
%matplotlib inline

### Load .csv files

In [2]:
# load default data frames
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
artists_df = pd.read_csv("artists.csv")
global_median_df = pd.read_csv("global_median.csv")
profiles = pd.read_csv("profiles.csv")

In [3]:
print len(train_df.user.unique())
print len(profiles.user.unique())

233286
233286


### Create profiles df, imput ages

In [4]:
# CREATES PROFILES DF

# # condition = (profiles_df['age'].isnull()) | (profiles_df['age'] < 95) | (profiles_df['age'] > 0)
# # profiles_df[condition] = np.nan
# profiles_df.loc[profiles_df["age"] < 5 ,'age'] = None
# profiles_df.loc[profiles_df["age"] > 85 ,'age'] = None
# profiles_df.loc[profiles_df["age"].isnull(),'age'] = None

# # calculate mean age per user
# mean_age = profiles_df[~profiles_df['age'].isnull()]["age"].mean()

# ###################################################################### 

# # calculate mean age per men
# males_only_df = profiles_df[profiles_df["sex"]=='m']
# mean_male_age = males_only_df[~males_only_df['sex'].isnull()]["age"].mean()

# ###################################################################### 

# # calculate mean age per women
# females_only_df = profiles_df[profiles_df["sex"]=='f']
# mean_female_age = females_only_df[~females_only_df['sex'].isnull()]["age"].mean()

# ###################################################################### 

# # get list of unique countries
# countries = profiles_df["country"].unique()

# # calculate mean age per country
# mean_country_age = {}
# for country in countries:
#     tmp_df = profiles_df[profiles_df["country"]==str(country)].copy()
#     mean_country_age[str(country)] = tmp_df["age"].mean()

# ###################################################################### 
    
# # calculate mean age per male per country
# mean_country_male_age = {}
# for country in countries:
#     tmp_df0 = profiles_df[profiles_df["sex"]=='m'].copy()
#     tmp_df1 = tmp_df0[profiles_df["country"]==str(country)].copy()
#     mean_country_male_age[str(country)] = tmp_df1["age"].mean()

# ###################################################################### 

# # calculate mean age per female per country
# mean_country_female_age = {}
# for country in countries:
#     tmp_df0 = profiles_df[profiles_df["sex"]=='f'].copy()
#     tmp_df1 = tmp_df0[profiles_df["country"]==str(country)].copy()
#     mean_country_female_age[str(country)] = tmp_df1["age"].mean()

# ###################################################################### 

# # initialize training df
# profiles = profiles_df

# ###################################################################### 

# # create sex indicators
# profiles["male"] = 0
# profiles["female"] = 0
# profiles["sex_missing"] = 0
# profiles.loc[profiles["sex"] =='m', "male"] = 1
# profiles.loc[profiles["sex"] =='f', "female"] = 1
# profiles.loc[profiles["sex"].isnull(), "sex_missing"] = 1

# ######################################################################  

# # create age indicators
# profiles["age_missing"] = 0
# profiles.loc[profiles["age"].isnull(),"age_missing"] = 1

# # fill in mean age: has no gender or country
# profiles.loc[profiles["age_missing"] == 1, "age"] = mean_age

# # fill in mean age: has gender, no country
# #males
# condition = (profiles["sex"] =='m') & (profiles["country"].isnull()) & (profiles["age_missing"] == 1)
# profiles.loc[condition, "age"] = mean_male_age
# # females
# condition = (profiles["sex"] =='f') & (profiles["country"].isnull()) & (profiles["age_missing"] == 1)
# profiles.loc[condition, "age"] = mean_female_age

# # fill in mean age: has country, no gender
# #loop over countries
# for country in countries:
#     condition = (profiles["country"] == country) & (profiles["age_missing"] == 1) & (profiles["sex_missing"] == 1)
#     profiles.loc[condition, "age"] = mean_country_age[country]

# # fill in mean age: has gender and country
# # loop over countries
# for country in countries:
#     # males
#     condition = (profiles["country"] == country) & (profiles["sex"] == 'm') & (profiles["age_missing"] == 1)
#     profiles.loc[condition, "age"] = mean_country_male_age[country]
#     # females
#     condition = (profiles["country"] == country) & (profiles["sex"] == 'f') & (profiles["age_missing"] == 1)
#     profiles.loc[condition, "age"] = mean_country_female_age[country]

# ###################################################################### 
    
# # create country indicators
# profiles["country_missing"] = 0
# profiles.loc[profiles["country"].isnull(), "country_missing"] = 1
# # loop over countries
# for country in countries:
#     # assign 1 if in that country
#     profiles.loc[profiles["country"] == country, country] = 1
#     # assign 0 otherwise
#     profiles.loc[profiles["country"] != country, country] = 0

# ###################################################################### 

# profiles.to_csv("profiles.csv")

# ###################################################################### 

# clean profiles df
profiles.drop("sex",inplace=True,axis=1)
profiles.drop("country",inplace=True,axis=1)
profiles.drop("Unnamed: 0",axis=1,inplace=True)
profiles.set_index("user",inplace=True)
profiles.drop("sex_missing",inplace=True,axis=1)
profiles.head(1)

,age,male,female,age_missing,country_missing,Sweden,Iceland,United States,Germany,Netherlands,...,Myanmar,Somalia,Gambia,Mozambique,Rwanda,Saint Kitts and Nevis,New Caledonia,Sudan,Guyana,Marshall Islands
user,,,,,,,,,,,,,,,,,,,,,
fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
profile_features = ["age","male","female","age_missing","United States","Germany","United Kingdom","France","Poland","Brazil","Spain","Italy"]
num_profile_features = len(profile_features)
sm_profiles = profiles.loc[:,profile_features]
sm_profiles.reset_index(inplace=True)
sm_profiles.head(1)

,user,age,male,female,age_missing,United States,Germany,United Kingdom,France,Poland,Brazil,Spain,Italy
0,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,0,1,0,0,0,0,0,0,0,0,0


### Grab genres for each artist

In [6]:
# load genre data frames and do some cleaning
artists_with_1genre_df = pd.read_csv("artists_with_genres.csv")
artists_with_5genres_df = pd.read_csv("artists_with_top_5_genres.csv")
artists_with_5genres_df.rename(columns = {"num_votes5":"extra"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre5":"num_votes5"}, inplace = True)
artists_with_5genres_df.rename(columns = {"num_votes4":"genre5"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre4":"num_votes4"}, inplace = True)
artists_with_5genres_df.rename(columns = {"num_votes3":"genre4"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre3":"num_votes3"}, inplace = True)
artists_with_5genres_df.rename(columns = {"num_votes2":"genre3"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre2":"num_votes2"}, inplace = True)
artists_with_5genres_df.rename(columns = {"num_votes1":"genre2"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre1":"num_votes1"}, inplace = True)
artists_with_5genres_df.rename(columns = {"name":"genre1"}, inplace = True)
artists_with_5genres_df.rename(columns = {"artist":"name"}, inplace = True)

# Calculate percent of votes
agn = artists_with_5genres_df[["num_votes1","num_votes2","num_votes3"]]
agn.rename(columns = {"num_votes1":"per_votes1"}, inplace = True)
agn.rename(columns = {"num_votes2":"per_votes2"}, inplace = True)
agn.rename(columns = {"num_votes3":"per_votes3"}, inplace = True)
agn = agn.apply(lambda c: c / c.sum() * 100, axis=1)

del artists_df

# combine dfs together
artists_with_5genres_df = pd.concat([artists_with_5genres_df, agn], axis=1)
artists_with_5genres_df.head(1)

,name,genre1,num_votes1,genre2,num_votes2,genre3,num_votes3,genre4,num_votes4,genre5,num_votes5,extra,per_votes1,per_votes2,per_votes3
03098741-08b3-4dd7-b3f6-1b0bfa2c879c,Liars,alternative rock,1,art punk,1,dance-punk,1,experimental rock,1,indie rock,1,NaN,33.333333,33.333333,33.333333


### Create train df

In [7]:
# Train - Test Split on profiles to ensure users are in one group or another
train0, test = train_test_split(train_df, test_size = .1)
train, valid = train_test_split(train0, test_size = .1)
#train = train_df.copy()

In [ ]:
# def genre_select(x,at5g):
#     artist = x[0]
#     genres = np.array([at5g.loc[artist,"genre1"],at5g.loc[artist,"genre2"],at5g.loc[artist,"genre3"]])
#     votes = np.array([at5g.loc[artist,"per_votes1"],at5g.loc[artist,"per_votes2"],at5g.loc[artist,"per_votes3"]])
#     votes_tot = np.sum(votes)
#     if votes_tot != 0:
#         weights = votes/float(votes_tot)
#         return np.random.choice(genres, p= weights)
#     else:
#         return None

In [8]:
#add primary genre to train
train["genre1"] = np.nan
train.set_index("artist", inplace=True)
train.update(artists_with_5genres_df)
train.reset_index(inplace=True)

test["genre1"] = np.nan
test.set_index("artist", inplace=True)
test.update(artists_with_5genres_df)
test.reset_index(inplace=True)

valid["genre1"] = np.nan
valid.set_index("artist", inplace=True)
valid.update(artists_with_5genres_df)
valid.reset_index(inplace=True)

train0["genre1"] = np.nan
train0.set_index("artist", inplace=True)
train0.update(artists_with_5genres_df)
train0.reset_index(inplace=True)

In [9]:
# train.genre = train.apply(lambda x: genre_select(x,artists_with_5genres_df),axis = 1)

In [10]:
# train.to_csv("train_wGenreChoice.csv")

### compute averages

In [11]:
# calculate global median
global_median = global_median_df.iloc[0,1]
del global_median_df
print global_median

118.0


In [12]:
# calculate user medians
user_medians = train.pivot(index='user',columns='artist',values='plays').reset_index().set_index('user').median(axis=1)
user_medians = pd.DataFrame(user_medians)
user_medians.rename(columns={0:"user median"},inplace=True)
user_medians.head(1)

,user median
user,
00000c289a1829a808ac09c00daf10bc3c4e223b,210


In [13]:
# calculate user means
user_means = train.pivot(index='user',columns='artist',values='plays').reset_index().set_index('user').mean(axis=1)
user_means = pd.DataFrame(user_means)
user_means.rename(columns={0:"user mean"},inplace=True)
user_means.head(1)

,user mean
user,
00000c289a1829a808ac09c00daf10bc3c4e223b,287.5


In [14]:
# calculate artist medians
artist_medians = train.pivot(index='user',columns='artist',values='plays').reset_index().set_index('user').median(axis=0)
artist_medians = pd.DataFrame(artist_medians)
artist_medians.rename(columns={0:"artist median"},inplace=True)
artist_medians.head(1)

,artist median
artist,
000d90ec-d64c-48a1-b775-e726fd240e9f,117


In [15]:
# calculate artist medians
artist_means = train.pivot(index='user',columns='artist',values='plays').reset_index().set_index('user').mean(axis=0)
artist_means = pd.DataFrame(artist_means)
artist_means.rename(columns={0:"artist mean"},inplace=True)
artist_means.head(1)

,artist mean
artist,
000d90ec-d64c-48a1-b775-e726fd240e9f,188.554187


In [16]:
# calculate genre medians
genre_medians = train.groupby("genre1").plays.median()
genre_medians = pd.DataFrame(genre_medians)
genre_medians.rename(columns={'plays':"genre median"},inplace=True)
genre_medians.head(1)

,genre median
genre1,
00s,92


In [17]:
# calculate genre mean
genre_means = train.groupby("genre1").plays.mean()
genre_means = pd.DataFrame(genre_means)
genre_means.rename(columns={'plays':"genre mean"},inplace=True)
genre_means.head(1)

,genre mean
genre1,
00s,211.392902


In [18]:
# calculate genre_user medians
user_genre_medians = train.groupby(["user","genre1"]).median().reset_index()
user_genre_medians.rename(columns={"plays": "user genre median"},inplace=True)
user_genre_medians.head(1)

,user,genre1,user genre median
0,00000c289a1829a808ac09c00daf10bc3c4e223b,acoustic rock,182


In [19]:
# calculate genre_user means
user_genre_means = train.groupby(["user","genre1"]).mean().reset_index()
user_genre_means.rename(columns={"plays": "user genre mean"},inplace=True)
user_genre_means.head(1)

,user,genre1,user genre mean
0,00000c289a1829a808ac09c00daf10bc3c4e223b,acoustic rock,182


# Combine Profile and Artist dfs

In [20]:
#pivot training data to have one row per user
train = train.pivot(index='user',columns='artist',values='plays').reset_index()
train = train.fillna(0)

In [ ]:
# FULL PROFILE
# join profiles and train together
# train.reset_index(inplace=True)
# train.set_index('user',inplace = True)
# train.drop("index",axis=1,inplace=True)

# profiles.reset_index(inplace=True)
# profiles.set_index('user', inplace=True)

# bigdf = pd.concat([profiles,train],axis=1)
# bigdf.fillna(0,inplace=True)
# bigdf.head(1)

In [21]:
# SMALL PROFILE
#join profiles and train together
train.set_index('user',inplace = True)
train.head(1)

artist,000d90ec-d64c-48a1-b775-e726fd240e9f,000fc734-b7e1-4a01-92d1-f544261b43f5,0019749d-ee29-4a5f-ab17-6bfa11deb969,0039c7ae-e1a7-4a7d-9b49-0cbc716821a6,004e5eed-e267-46ea-b504-54526f1f377d,00565b31-14a3-4913-bd22-385eb40dd13c,00a9f935-ba93-4fc8-a33a-993abe9c936b,00eeed6b-5897-4359-8347-b8cd28375331,0103c1cc-4a09-4a5d-a344-56ad99a77193,0110e63e-0a9b-4818-af8e-41e180c20b9a,...,ff6e677f-91dd-4986-a174-8db0474b1799,ff7f80cd-05c2-4068-a00e-fbfbd453d049,ff865aa0-4603-4f79-ae8b-8735332e2cfa,ff95eb47-41c4-4f7f-a104-cdc30f02e872,ff9deaae-da4f-42b7-a19e-36fedd3fc706,ffb18e19-64a4-4a65-b4ce-979e00c3c69d,ffb2d3e3-a4cc-48cf-8fb0-f2f846e9d7b9,ffb390b8-8df4-4b72-97d1-7b2fc008a452,ffe16bba-4d84-409b-8f22-5242c60b930f,ffe9ec08-6b6b-4993-9394-e280b429dbfd
user,,,,,,,,,,,,,,,,,,,,,
00000c289a1829a808ac09c00daf10bc3c4e223b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# get index for later
users = sm_profiles.user
sm_profiles.set_index('user', inplace=True)
# fill in any missing values just in case
sm_profiles[sm_profiles.age.isnull()]=sm_profiles.age.mean()

# standardize profile data
scaler = StandardScaler()
sm_profiles_scaled = pd.DataFrame(scaler.fit_transform(sm_profiles))
cols = sm_profiles.columns
sm_profiles_scaled.columns = cols
sm_profiles_scaled.index = users

# combine profile and training data
bigdf = pd.concat([sm_profiles_scaled,train],axis=1,join_axes=[sm_profiles_scaled.index])
bigdf.head(1)

,age,male,female,age_missing,United States,Germany,United Kingdom,France,Poland,Brazil,...,ff6e677f-91dd-4986-a174-8db0474b1799,ff7f80cd-05c2-4068-a00e-fbfbd453d049,ff865aa0-4603-4f79-ae8b-8735332e2cfa,ff95eb47-41c4-4f7f-a104-cdc30f02e872,ff9deaae-da4f-42b7-a19e-36fedd3fc706,ffb18e19-64a4-4a65-b4ce-979e00c3c69d,ffb2d3e3-a4cc-48cf-8fb0-f2f846e9d7b9,ffb390b8-8df4-4b72-97d1-7b2fc008a452,ffe16bba-4d84-409b-8f22-5242c60b930f,ffe9ec08-6b6b-4993-9394-e280b429dbfd
user,,,,,,,,,,,,,,,,,,,,,
fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,0.079937,-1.282467,1.533488,-0.442665,-0.460488,-0.25027,-0.271015,-0.081249,-0.173638,-0.145808,...,0,0,0,0,0,0,0,0,0,0


# KNN

In [109]:
bigdf.fillna(0,inplace=True)
neigh = KNeighborsRegressor(n_neighbors=1000, weights="uniform")
%time neigh.fit(bigdf.iloc[:,:num_profile_features],bigdf.iloc[:,num_profile_features:])

CPU times: user 2min 18s, sys: 201 ms, total: 2min 18s
Wall time: 2min 18s


KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=1000, p=2,
          weights='uniform')

In [149]:
# # use this function in APPLY for a df
# def knn_profile_predict(test_row):

#     artist = test_row[0]
#     user = test_row[1]  

#     to_test = bigdf.iloc[bigdf.index == user,:num_profile_features]

#     prediction = 0
#     counter = 0
#     while (prediction == 0) & (counter < 100):
#         neighbor_idx = neigh.kneighbors(to_test)[1][0][counter]
#         prediction = bigdf.ix[neighbor_idx,artist]
#         counter +=1
#     return prediction

In [192]:
# # use this function in APPLY for a df
# def knn_profile_predict(test_row):

#     artist = test_row[0]
#     user = test_row[1]  

#     to_test = bigdf.iloc[bigdf.index == user,:num_profile_features]

#     prediction = 0
#     predictions = []
#     counter = 0
#     while (len(np.array(predictions)[np.nonzero(predictions)])< 3) & (counter < 500):
#         neighbor_idx = neigh.kneighbors(to_test)[1][0][counter]
#         predictions.append(bigdf.ix[neighbor_idx,artist])
#         counter +=1
    
#     return np.median(np.array(predictions)[np.nonzero(predictions)])

In [236]:
# use this function in APPLY for a df
def knn_profile_predict(test_row,k):

    artist = test_row[0]
    user = test_row[1]  

    to_test = bigdf.iloc[bigdf.index == user,:num_profile_features]

    prediction = 0
    predictions = []
    counter = 0
    
    idx = neigh.kneighbors(to_test)[1][0]
    predictions = np.array([bigdf.ix[iix,artist] for iix in idx])
    return np.median(predictions[np.nonzero(predictions)][:k])

In [ ]:
valid["knn pred"] = None
%time valid["knn1 pred"] = valid.iloc[:10000,:].apply(lambda x: knn_profile_predict(x,1),axis=1)

In [227]:
valid.head(1000)

,artist,user,plays,genre1,kmeans pred,knn pred,AE
0,6b49dc39-cb6f-465f-980b-bc17a271da34,05c2257e7da092b52b15e0ad61fb7763414430ea,120,austrian,0,78.0,42.0
1,d5d97b2b-b83b-4976-814a-056d9076c8c3,bef2303bd0fae8baf8e22e296979e38cd9c33b1d,193,east coast hip hop,0,131.0,62.0
2,b88ca659-0393-4a62-abd8-f290e6c7a7e2,d0408aa10cffcff3a0e834d3d819e5406b32934c,353,soundtrack,NaN,100.0,253.0
3,0881daf1-20df-4a3e-a84f-6476a84bb172,16fa8dc6da7e459e3f1e1f7a0f7b6e9d6f0422a3,142,british,NaN,85.0,57.0
4,b6b2bb8d-54a9-491f-9607-7b546023b433,d9f7a9d1e72fcfb84de0269ae63cc72014e45341,102,alternative rock,NaN,146.0,44.0
5,8f9d6bb2-dba4-4cca-9967-cc02b9f4820c,86728c8fdc265e7c1c227c1ea6524755caf92be5,68,rapcore,NaN,182.5,114.5
6,cc0b7089-c08d-4c10-b6b0-873582c17fd6,b028e2c8d296482847e43716008fbbc7a148f86a,82,alternative metal,NaN,161.0,79.0
7,d2ff6b6b-fc30-48dc-8952-06f9d8fc64f8,2d6f1d6e89b45773f07c8ab2f999ef518d92cc2a,26,progressive rock,NaN,85.0,59.0
8,14b22b4b-06d5-4b82-8284-29d29b58945f,8a771dfcbe0eedacd7370cd97886ebb800a7423d,37,NaN,NaN,113.0,76.0
9,76c9a186-75bd-436a-85c0-823e3efddb7f,868364cd542905a402ff2fc3f160985fdcdd08e9,357,blues rock,NaN,83.0,274.0


# KMEANS

In [ ]:
# # standardize columns before kmeans
# scaler = StandardScaler()
# bigdf_sc = scaler.fit_transform(bigdf)

In [ ]:
# # carry out kmeans
# kmeans = KMeans(init = 'k-means++', n_clusters=50, n_jobs = -1, n_init = 8)
# kmeans.fit(bigdf_sc)

In [ ]:
# from sklearn.externals import joblib
# joblib.dump(kmeans, 'kmeans50.pkl') 

In [ ]:
# cols = bigdf.columns
# centers = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_))
# centers.columns = cols
# centers.head()

### Create test array to compare to kmeans

In [ ]:
closest_centers = kmeans.predict(bigdf_sc)
train['closest center'] = closest_centers.tolist()
test['closest center'] = None
test.set_index("user",inplace=True)
test.update(train)
test.reset_index(inplace=True)

In [ ]:
def get_prediction(x):
    artist = x[1]
    return centers.ix[x[5],artist]

In [ ]:
test["kmeans pred"] = 0
test["kmeans pred"] = test.apply(lambda x: get_prediction(x),axis=1)
test.head()

### Carry out ensemble

In [ ]:
# initialize columns
test['user genre median'] = np.nan
test['user median'] = np.nan
test['artist median'] = np.nan
test['genre median'] = np.nan
test['global median'] = np.nan
test['user mean'] = np.nan
test['genre mean'] = np.nan
test['artist mean'] = np.nan
test['user genre mean'] = np.nan

# add global median
test['global median'] = global_median

# add user median
test.set_index('user',inplace=True)
test.update(user_medians)
test.reset_index(inplace=True)

# add artist median
test.set_index('artist',inplace=True)
test.update(artist_medians)
test.reset_index(inplace=True)
test["genre median"].fillna(test["user median"],inplace=True)

# add genre median
test.set_index('genre1',inplace=True)
test.update(genre_medians)
test.reset_index(inplace=True)
test["genre median"].fillna(test["user median"],inplace=True)

# add user genre median
test.set_index(['user','genre1'],inplace=True)
test.update(user_genre_medians.set_index(["user","genre1"]))
test.reset_index(inplace=True)
test["user genre median"].fillna(test["user median"],inplace=True)

# add user mean
test.set_index('user',inplace=True)
test.update(user_means)
test.reset_index(inplace=True)

# add artist mean
test.set_index('artist',inplace=True)
test.update(artist_means)
test.reset_index(inplace=True)
test["genre mean"].fillna(test["user mean"],inplace=True)

# add genre mean
test.set_index('genre1',inplace=True)
test.update(genre_means)
test.reset_index(inplace=True)
test["genre mean"].fillna(test["user mean"],inplace=True)

# add user genre mean
test.set_index(['user','genre1'],inplace=True)
test.update(user_genre_means.set_index(["user","genre1"]))
test.reset_index(inplace=True)
test["user genre mean"].fillna(test["user mean"],inplace=True)
test.head(1)

In [ ]:
# weighted ensemble
stacking= linear_model.LinearRegression()

X = np.vstack([test['kmeans pred'].values,test['user median'].values])
X = np.vstack([X,test['user genre median'].values])
X = np.vstack([X,test['genre median'].values])
X = np.vstack([X,test['artist median'].values])
X = np.vstack([X,test['global median'].values])
X = np.vstack([X,test['user mean'].values])
X = np.vstack([X,test['user genre mean'].values])
X = np.vstack([X,test['genre mean'].values])
X = np.vstack([X,test['artist mean'].values])
X = X.T
X.shape

y = test['plays'].values

stacking.fit(X,y)
stacking.coef_

In [ ]:
test['ensemble'] =                            \
stacking.coef_[0]*test['kmeans pred']         \
+ stacking.coef_[1]*test['user median']       \
+ stacking.coef_[2]*test['user genre median'] \
+ stacking.coef_[3]*test['genre median']      \
+ stacking.coef_[4]*test['artist median']     \
+ stacking.coef_[5]*test['global median']     \
+ stacking.coef_[6]*test['user mean']
+ stacking.coef_[7]*test['user genre mean']   \
+ stacking.coef_[8]*test['genre mean']        \
+ stacking.coef_[9]*test['artist mean']      

#test['AE'] = np.abs(test['plays']-test['ensemble'])  

In [ ]:
#print "MAE is:", test.AE.mean()

In [ ]:
output_for_kaggle = test
output_for_kaggle.drop(["plays","closest center", "AE", "genre1","user","artist","kmeans pred","user genre median","user median", "genre median", "global median","artist median","artist mean","user mean", "genre mean", "user genre mean"],axis=1,inplace=True)
output_for_kaggle.rename(columns={"ensemble":"plays"},inplace = True)
output_for_kaggle.head()

In [ ]:
train_mf = train.drop("closest center",axis = 1)
train_mf.fillna(0,inplace=True)
train_mf.head(1)

In [ ]:
model = NMF(init='nndsvd')
model.fit(train_mf) 

In [ ]:
joblib.dump(model, 'matrixfactorization.pkl') 

In [84]:
test_row = valid.iloc[0,:]

artist = test_row[0]
user = test_row[1]  

to_test = bigdf.iloc[bigdf.index == user,:num_profile_features]

prediction = 0
counter = 0
while (prediction == 0) & (counter < 10):
    neighbor_idx = neigh.kneighbors(to_test)[1][0][counter]
    print neighbor_idx
    prediction = bigdf.ix[neighbor_idx,artist]
    print prediction
    counter +=1

76419
0.0
85331
0.0
8827
0.0
29844
0.0
79874
0.0
36976
0.0
29838
0.0
55180
0.0
91312
0.0
35916
0.0


In [53]:
neighbor_idx = neigh.kneighbors(to_test)[1][0][0]

In [54]:
test_row = valid.iloc[0,:]

In [55]:
test_row[0]

'6b49dc39-cb6f-465f-980b-bc17a271da34'

In [56]:
test_row[1] 

'05c2257e7da092b52b15e0ad61fb7763414430ea'

In [57]:
bigdf.iloc[bigdf.index == user,:num_profile_features]

,age,male,female,age_missing,United States,Germany,United Kingdom,France,Poland,Brazil,Spain,Italy
user,,,,,,,,,,,,
05c2257e7da092b52b15e0ad61fb7763414430ea,0.56875,0.650253,-0.528804,-0.442665,-0.460488,-0.25027,-0.271015,-0.081249,-0.173638,-0.145808,-0.130617,-0.083197


In [82]:
neighbor_idx = neigh.kneighbors(to_test)[1][0][10]

In [83]:
bigdf.ix[neighbor_idx,artist]

0.0